In [585]:
# !wget https://datahack-prod.s3.amazonaws.com/train_file/train_0OECtn8.csv
# !wget https://datahack-prod.s3.amazonaws.com/test_file/test_1zqHu22.csv

In [586]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

In [587]:
raw_train = pd.read_csv("train_0OECtn8.csv")
raw_test = pd.read_csv("test_1zqHu22.csv")

In [588]:
print(f"Train shape {raw_train.shape}, Test shape {raw_test.shape}")

Train shape (89197, 10), Test shape (11121, 9)


In [589]:
raw_train.head(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35


In [590]:
raw_train.tail(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
89194,89195,13655,16,97,25,Male,Student,270,462,4.23
89195,89196,24840,9,18,35,Male,Working Professional,230,819,3.77
89196,89197,27183,25,150,13,Male,Student,240,317,4.31


In [591]:
raw_test.head(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views
0,89198,7986,12,42,14,Male,Student,180,138
1,89199,11278,34,115,14,Male,Student,230,840
2,89200,17245,8,110,44,Female,Working Professional,280,628


In [592]:
raw_test.tail(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views
11118,100316,2042,16,98,22,Male,Student,270,462
11119,100317,24626,8,16,33,Male,Other,280,628
11120,100318,967,8,74,33,Male,Working Professional,280,628


In [593]:
raw_train.isna().sum()

row_id              0
user_id             0
category_id         0
video_id            0
age                 0
gender              0
profession          0
followers           0
views               0
engagement_score    0
dtype: int64

In [594]:
raw_test.isna().sum()

row_id         0
user_id        0
category_id    0
video_id       0
age            0
gender         0
profession     0
followers      0
views          0
dtype: int64

In [595]:
raw_train.describe()

,row_id,user_id,category_id,video_id,age,followers,views,engagement_score
count,89197.000000,89197.000000,89197.000000,89197.000000,89197.000000,89197.000000,89197.000000,89197.000000
mean,44599.000000,13881.909806,18.323733,77.715383,24.848616,252.460172,502.980268,3.487797
std,25749.100318,8005.582771,11.675154,48.469656,8.955535,46.094468,268.569482,0.863498
min,1.000000,1.000000,1.000000,1.000000,10.000000,160.000000,30.000000,0.000000
25%,22300.000000,6945.000000,8.000000,34.000000,18.000000,230.000000,229.000000,2.900000
50%,44599.000000,13892.000000,16.000000,76.000000,23.000000,240.000000,467.000000,3.710000
75%,66898.000000,20819.000000,26.000000,120.000000,32.000000,280.000000,714.000000,4.150000
max,89197.000000,27734.000000,47.000000,175.000000,68.000000,360.000000,1000.000000,5.000000


In [596]:
raw_train.describe(include=np.object)

,gender,profession
count,89197,89197
unique,2,3
top,Male,Student
freq,52397,44638


In [597]:
raw_train['user_id'].value_counts().min(),raw_train['user_id'].value_counts().max()

(2, 10)

In [598]:
raw_train['user_id'].value_counts().head(3)

5198     10
13218    10
1448     10
Name: user_id, dtype: int64

In [599]:
raw_train[raw_train['user_id']==5198]

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
22708,22709,5198,5,56,32,Male,Other,240,229,4.30
28089,28090,5198,5,79,32,Male,Other,240,229,4.60
32687,32688,5198,5,9,32,Male,Other,240,229,4.50
33424,33425,5198,5,169,32,Male,Other,240,229,4.28
35305,35306,5198,5,163,32,Male,Other,240,229,4.26
48919,48920,5198,5,161,32,Male,Other,240,229,4.47
62534,62535,5198,5,69,32,Male,Other,240,229,4.13
73528,73529,5198,5,90,32,Male,Other,240,229,4.44
79107,79108,5198,5,10,32,Male,Other,240,229,4.51
85219,85220,5198,5,155,32,Male,Other,240,229,4.44


In [600]:
common_user_id = set(raw_train['user_id'].unique()).intersection(set(raw_test['user_id'].unique()))
len(common_user_id)

10384

In [601]:
raw_train['user_id'].nunique(), raw_test['user_id'].nunique()

(27734, 10384)

In [602]:
raw_train['category_id'].value_counts(normalize=True).sort_index()

1     0.020292
2     0.001872
3     0.020685
4     0.041235
5     0.090855
6     0.015684
7     0.021133
8     0.070776
9     0.021144
10    0.013644
11    0.042221
12    0.043600
13    0.031133
14    0.006009
15    0.024261
16    0.036593
17    0.013543
18    0.025124
19    0.052457
20    0.007870
21    0.045327
22    0.004103
23    0.029945
24    0.005875
25    0.054710
26    0.012960
27    0.012366
28    0.014675
29    0.010269
30    0.006144
31    0.020797
32    0.030405
33    0.005796
34    0.036963
35    0.011054
36    0.011491
37    0.010785
38    0.005987
39    0.015012
40    0.005135
41    0.006110
42    0.024418
43    0.011424
44    0.002399
45    0.002545
46    0.002646
47    0.000527
Name: category_id, dtype: float64

In [603]:
raw_train['video_id'].value_counts(normalize=False).head(10)

112    1337
53     1334
1      1282
65     1103
42     1077
46      938
4       932
10      921
5       913
87      902
Name: video_id, dtype: int64

In [604]:
raw_train[raw_train['video_id']==10]

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
115,116,476,5,10,21,Female,Student,240,229,2.48
181,182,16877,5,10,29,Female,Other,240,229,3.22
214,215,13064,5,10,33,Female,Other,240,229,2.30
357,358,25281,5,10,16,Male,Student,240,229,3.80
399,400,3718,5,10,14,Female,Student,240,229,4.57
...,...,...,...,...,...,...,...,...,...,...
88844,88845,16113,5,10,15,Female,Student,240,229,2.80
88901,88902,21712,5,10,24,Male,Other,240,229,3.64
89066,89067,17572,5,10,13,Male,Student,240,229,3.62
89133,89134,26217,5,10,20,Female,Student,240,229,2.19


In [605]:
raw_test[raw_test['video_id']==10]

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views
51,89249,4033,5,10,26,Female,Working Professional,240,229
92,89290,25610,5,10,40,Female,Working Professional,240,229
285,89483,19113,5,10,15,Female,Student,240,229
362,89560,15928,5,10,25,Female,Other,240,229
371,89569,8313,5,10,25,Male,Other,240,229
...,...,...,...,...,...,...,...,...,...
10901,100099,5556,5,10,22,Female,Working Professional,240,229
11013,100211,8865,5,10,32,Female,Working Professional,240,229
11039,100237,18756,5,10,35,Male,Other,240,229
11050,100248,4483,5,10,36,Female,Other,240,229


In [606]:
raw_test[raw_test['video_id']==1].sort_values(by='user_id')

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views
56,89254,16,19,1,16,Female,Student,230,369
5032,94230,78,1,1,18,Male,Student,360,990
9168,98366,362,19,1,31,Male,Other,230,369
9474,98672,1041,19,1,16,Female,Student,230,369
5133,94331,1080,19,1,26,Male,Other,230,369
...,...,...,...,...,...,...,...,...,...
8440,97638,27347,1,1,32,Male,Other,360,990
7058,96256,27390,1,1,13,Female,Student,360,990
1418,90616,27479,19,1,36,Female,Working Professional,230,369
9950,99148,27501,19,1,14,Female,Student,230,369


In [607]:
raw_train[raw_train['video_id']==1].sort_values(by='user_id')

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
30684,30685,1,1,1,18,Female,Student,360,990,2.88
82236,82237,10,1,1,26,Female,Student,360,990,2.70
1105,1106,36,19,1,15,Male,Student,230,369,4.25
10818,10819,51,19,1,18,Female,Student,230,369,4.29
43691,43692,56,1,1,33,Female,Other,360,990,2.30
...,...,...,...,...,...,...,...,...,...,...
68490,68491,27663,1,1,37,Male,Working Professional,360,990,3.32
51117,51118,27669,19,1,18,Male,Student,230,369,4.21
34829,34830,27691,19,1,30,Male,Other,230,369,3.81
75844,75845,27696,1,1,22,Female,Other,360,990,4.25


In [608]:
raw_train.groupby(['video_id'])[['followers','views','category_id']].std()

,followers,views,category_id
video_id,,,
1,64.827241,309.674744,8.97608
2,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.00000
5,0.000000,0.000000,0.00000
...,...,...,...
171,0.000000,0.000000,0.00000
172,0.000000,0.000000,0.00000
173,0.000000,0.000000,0.00000


In [609]:
raw_train.groupby(['user_id'])[['followers','views','category_id']].std()

,followers,views,category_id
user_id,,,
1,80.829038,554.256258,0.577350
2,21.908902,87.635609,0.547723
3,17.320508,76.787586,0.577350
4,115.470054,301.954191,3.464102
5,28.867513,110.273901,0.577350
...,...,...,...
27730,57.735027,72.746134,1.154701
27731,14.142136,144.249783,1.414214
27732,0.000000,0.000000,0.000000


In [610]:
raw_train['video_id'].nunique(),raw_test['video_id'].nunique()

(175, 128)

In [611]:
common_video_id = set(raw_train['video_id'].unique()).intersection(set(raw_test['video_id'].unique()))
len(common_video_id)

128

In [612]:
raw_train.groupby(['video_id'])['category_id'].nunique().describe()

count    175.000000
mean       1.005714
std        0.075593
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
Name: category_id, dtype: float64

In [613]:
raw_train.groupby(['user_id'])['profession'].nunique().describe()

count    27734.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: profession, dtype: float64

In [614]:
df = pd.concat([raw_train,raw_test],axis=0,ignore_index=True,sort=False)
df.shape[0] == raw_train.shape[0]+raw_test.shape[0]

True

In [615]:
df.sample(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
97036,97037,26211,16,137,32,Female,Other,270,462,NaN
57637,57638,11296,34,95,18,Female,Student,230,840,2.88
60525,60526,21482,5,87,13,Female,Student,240,229,4.39


In [616]:
df['category_id'].nunique(),df['video_id'].nunique()

(47, 175)

In [617]:
def cv(x):
    return np.mean(x)/np.mean(0.001+np.std(x))

In [618]:
user_profile = df.groupby(['user_id']).agg({'category_id':pd.Series.nunique,
                                            'video_id':pd.Series.nunique,
                                            'views':['mean','sum','std','count'],
                                            'followers':['mean','sum','std'],
                                            # 'engagement_score': cv,
                                            })

user_profile.columns = ["_".join(x) for x in user_profile.columns]
user_profile =user_profile.add_suffix("_usr")
user_profile = user_profile.reset_index()
user_profile.describe()

,user_id,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr
count,27734.000000,27734.000000,27734.000000,27734.000000,27734.000000,27734.000000,27734.000000,27734.000000,27734.000000,27734.000000
mean,13867.500000,1.798875,3.617149,509.632520,1800.098832,128.891739,3.617149,251.409215,912.075791,21.791368
std,8006.260519,0.589336,1.451445,222.416486,955.171705,133.947188,1.451445,39.179037,390.636733,24.240826
min,1.000000,1.000000,2.000000,44.000000,88.000000,0.000000,2.000000,160.000000,320.000000,0.000000
25%,6934.250000,1.000000,3.000000,329.333333,1182.000000,0.000000,3.000000,230.000000,650.000000,0.000000
50%,13867.500000,2.000000,3.000000,514.333333,1753.000000,92.953393,3.000000,243.333333,840.000000,17.320508
75%,20800.750000,2.000000,4.000000,692.333333,2287.000000,209.000797,4.000000,276.000000,1127.500000,34.641016
max,27734.000000,3.000000,12.000000,1000.000000,8190.000000,678.822510,12.000000,360.000000,3220.000000,141.421356


In [619]:
user_profile.sample(3)

,user_id,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr
15430,15431,1,2,138.0,276,0.000000,2,180.000000,360,0.000000
14295,14296,2,3,714.0,2142,218.238402,3,243.333333,730,23.094011
1811,1812,1,5,229.0,1145,0.000000,5,240.000000,1200,0.000000


In [620]:
df.groupby(['user_id'])['age'].std().max()

0.0

In [621]:
df.groupby(['user_id'])['followers'].std().max()

141.4213562373095

In [622]:
cat_id = df.groupby(['category_id']).agg({'user_id':pd.Series.nunique,
                                          'video_id':pd.Series.nunique,
                                          'views':['mean','sum','count'],
                                          'followers':['mean','sum'],
                                           'engagement_score': cv,
                                          })

cat_id.columns = ["_".join(x) for x in cat_id.columns]
cat_id = cat_id.add_suffix("_cat")
cat_id = cat_id.reset_index()

cat_id.describe()


,category_id,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat
count,47.000000,47.000000,47.000000,47.000000,4.700000e+01,47.000000,47.000000,4.700000e+01,47.000000
mean,24.000000,1061.489362,3.744681,516.425532,1.062212e+06,2134.425532,247.872340,5.382023e+05,4.272305
std,13.711309,730.989469,4.613157,299.915767,1.114549e+06,2078.421169,45.105488,5.347228e+05,0.811530
min,1.000000,47.000000,1.000000,30.000000,5.010000e+03,47.000000,160.000000,1.222000e+04,2.589629
25%,12.500000,529.000000,2.000000,238.500000,2.360070e+05,546.500000,220.000000,1.351950e+05,3.821286
50%,24.000000,869.000000,2.000000,613.000000,6.261840e+05,1321.000000,240.000000,3.270400e+05,4.228355
75%,35.500000,1588.500000,4.500000,793.500000,1.656089e+06,2792.000000,270.000000,7.938600e+05,4.737077
max,47.000000,3043.000000,31.000000,1000.000000,4.828064e+06,9593.000000,360.000000,2.302320e+06,5.981356


In [623]:
cat_id.sample(4)

,category_id,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat
24,25,2382,8,317.0,1851597,5841,240.0,1401840,3.564623
41,42,1251,3,689.0,1628796,2364,290.0,685560,3.431949
23,24,524,1,617.0,323308,524,200.0,104800,2.928793
33,34,1679,5,840.0,3342360,3979,230.0,915170,2.665469


In [624]:
video_id = df.groupby(['video_id']).agg({'user_id':pd.Series.nunique,
                                         'views':['mean','sum','count'],
                                         'followers':['mean','sum'],
                                         'engagement_score': cv,
                                         })
video_id.columns = ["_".join(x) for x in video_id.columns]
video_id = video_id.add_suffix("_vid")
video_id = video_id.reset_index()


video_id.describe()

,video_id,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,88.000000,573.245714,462.959389,285279.662857,573.245714,248.894282,144545.771429,4.361403
std,50.662281,293.527970,282.121367,227969.966162,293.527970,42.481791,79718.247823,0.753619
min,1.000000,47.000000,30.000000,4928.000000,47.000000,160.000000,12220.000000,2.481437
25%,44.500000,313.000000,229.000000,51558.000000,313.000000,230.000000,76015.000000,3.938282
50%,88.000000,642.000000,414.000000,254848.000000,642.000000,240.000000,144900.000000,4.353067
75%,131.500000,787.000000,689.000000,476652.000000,787.000000,270.000000,207500.000000,4.736710
max,175.000000,1514.000000,1000.000000,967284.000000,1514.000000,360.000000,433760.000000,7.153651


In [625]:
video_id.sample(3)

,video_id,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid
0,1,1514,638.892999,967284,1514,286.499339,433760,4.570742
154,155,218,229.000000,49922,218,240.000000,52320,4.420753
135,136,758,689.000000,522262,758,290.000000,219820,3.385609


In [626]:
usr_cat = df.groupby(['user_id','category_id']).agg({'video_id':pd.Series.nunique,
                                                     'views':['mean','sum','count',cv],
                                                     'followers':['mean','sum',cv],
                                                    # 'engagement_score': cv,
                                                     })
usr_cat.columns = ["_".join(x) for x in usr_cat.columns]
usr_cat = usr_cat.add_suffix("_usr_cat")
usr_cat = usr_cat.reset_index()


usr_cat.describe()

,user_id,category_id,video_id_nunique_usr_cat,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,views_cv_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat,followers_cv_usr_cat
count,49890.000000,49890.000000,49890.000000,49890.000000,49890.000000,49890.000000,49890.000000,49890.000000,49890.000000,49890.000000
mean,13860.321227,19.483664,2.010784,520.871177,1000.680317,2.010784,520871.176588,253.863299,507.025656,253863.299258
std,8003.431271,11.855247,1.082500,269.311005,666.771010,1.082500,269311.004876,47.628561,278.288983,47628.560540
min,1.000000,1.000000,1.000000,30.000000,30.000000,1.000000,30000.000000,160.000000,160.000000,160000.000000
25%,6931.250000,9.000000,1.000000,309.000000,462.000000,1.000000,309000.000000,230.000000,280.000000,230000.000000
50%,13844.000000,18.000000,2.000000,613.000000,884.000000,2.000000,613000.000000,240.000000,460.000000,240000.000000
75%,20792.750000,28.000000,2.000000,714.000000,1378.000000,2.000000,714000.000000,280.000000,660.000000,280000.000000
max,27734.000000,47.000000,11.000000,1000.000000,4200.000000,11.000000,1000000.000000,360.000000,2640.000000,360000.000000


In [627]:
vid_cat = df.groupby(['video_id','category_id']).agg({'user_id':pd.Series.nunique,
                                                     'views':['mean','sum','count'],
                                                     'followers':['mean','sum',],
                                                    #  'engagement_score': cv,
                                                     })
vid_cat.columns = ["_".join(x) for x in vid_cat.columns]
vid_cat = vid_cat.add_suffix("_usr_cat")
vid_cat = vid_cat.reset_index()


vid_cat.describe()

,video_id,category_id,user_id_nunique_usr_cat,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat
count,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000
mean,87.505682,18.676136,569.988636,464.420455,283658.755682,569.988636,249.204545,143724.488636
std,50.941198,12.955868,284.774156,283.894959,223123.732137,284.774156,43.113713,76840.120865
min,1.000000,1.000000,47.000000,30.000000,4928.000000,47.000000,160.000000,12220.000000
25%,43.750000,6.750000,315.000000,229.000000,51831.000000,315.000000,230.000000,76417.500000
50%,87.500000,16.000000,643.000000,414.000000,257171.000000,643.000000,240.000000,145425.000000
75%,131.250000,29.250000,786.500000,689.000000,470372.000000,786.500000,270.000000,207250.000000
max,175.000000,47.000000,1337.000000,1000.000000,819581.000000,1337.000000,360.000000,352960.000000


In [628]:
all_cat = df.groupby(['user_id','video_id','category_id']).agg({
                                                        'views':['mean','sum','count'],
                                                     'followers':['mean','sum',],
                                                     #'engagement_score': cv,
                                                     })
all_cat.columns = ["_".join(x) for x in all_cat.columns]
all_cat = all_cat.add_suffix("_usr_cat")
all_cat = all_cat.reset_index()


all_cat.describe()

,user_id,video_id,category_id,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat
count,100318.000000,100318.000000,100318.000000,100318.000000,100318.000000,100318.0,100318.000000,100318.000000
mean,13875.679350,77.940110,18.029157,497.656861,497.656861,1.0,252.153253,252.153253
std,8005.079041,48.499456,11.562197,266.974474,266.974474,0.0,45.324580,45.324580
min,1.000000,1.000000,1.000000,30.000000,30.000000,1.0,160.000000,160.000000
25%,6938.250000,35.000000,8.000000,229.000000,229.000000,1.0,230.000000,230.000000
50%,13889.000000,76.000000,16.000000,467.000000,467.000000,1.0,240.000000,240.000000
75%,20813.000000,121.000000,26.000000,709.000000,709.000000,1.0,280.000000,280.000000
max,27734.000000,175.000000,47.000000,1000.000000,1000.000000,1.0,360.000000,360.000000


In [629]:
raw_train.head(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35


In [630]:
featured_df = df.copy()
featured_df = featured_df.merge(user_profile,how='left',on=['user_id'])
featured_df = featured_df.merge(cat_id,how='left',on=['category_id'])
featured_df = featured_df.merge(video_id,how='left',on=['video_id'])
featured_df = featured_df.merge(usr_cat,how='left',on=['user_id','category_id'])
featured_df = featured_df.merge(vid_cat,how='left',on=['video_id','category_id'])
featured_df = featured_df.merge(all_cat,how='left',on=['user_id','category_id','video_id'])

featured_df.head(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid,video_id_nunique_usr_cat,views_mean_usr_cat_x,views_sum_usr_cat_x,views_count_usr_cat_x,views_cv_usr_cat,followers_mean_usr_cat_x,followers_sum_usr_cat_x,followers_cv_usr_cat,user_id_nunique_usr_cat,views_mean_usr_cat_y,views_sum_usr_cat_y,views_count_usr_cat_y,followers_mean_usr_cat_y,followers_sum_usr_cat_y,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat
0,1,19990,37,128,24,Male,Student,180,1000,4.33,2,3,744.666667,2234,221.125153,3,246.666667,740,57.735027,635,2,1000.0,962000,962,180.0,173160,3.618930,503,1000.0,503000,503,180.0,90540,3.654689,1,1000.0,1000,1,1000000.0,180.0,180,180000.0,503,1000.0,503000,503,180.0,90540,1000.0,1000,1,180.0,180
1,2,5304,32,132,14,Female,Student,330,714,1.79,3,5,602.800000,3014,130.273558,5,308.000000,1540,34.928498,1507,4,714.0,2239818,3137,330.0,1035210,3.994371,807,714.0,576198,807,330.0,266310,4.005123,2,714.0,1428,2,714000.0,330.0,660,330000.0,807,714.0,576198,807,330.0,266310,714.0,714,1,330.0,330
2,3,1840,12,24,19,Male,Student,180,138,4.35,1,4,138.000000,552,0.000000,4,180.000000,720,0.000000,2020,6,138.0,608856,4412,180.0,794160,4.913325,644,138.0,88872,644,180.0,115920,4.602973,4,138.0,552,4,138000.0,180.0,720,180000.0,644,138.0,88872,644,180.0,115920,138.0,138,1,180.0,180


In [631]:
featured_df['profession'].unique()

array(['Student', 'Working Professional', 'Other'], dtype=object)

In [632]:
featured_df['male'] = (featured_df['gender'] == 'Male').astype(int)
featured_df['student'] = (featured_df['profession'] == 'Student').astype(int)
featured_df['working'] = (featured_df['profession'] == 'Working Professional').astype(int)
featured_df.sample(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid,video_id_nunique_usr_cat,views_mean_usr_cat_x,views_sum_usr_cat_x,views_count_usr_cat_x,views_cv_usr_cat,followers_mean_usr_cat_x,followers_sum_usr_cat_x,followers_cv_usr_cat,user_id_nunique_usr_cat,views_mean_usr_cat_y,views_sum_usr_cat_y,views_count_usr_cat_y,followers_mean_usr_cat_y,followers_sum_usr_cat_y,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat,male,student,working
15434,15435,6736,27,65,38,Male,Other,320,309,3.26,2,5,205.8,1029,57.690554,5,264.0,1320,31.304952,1103,1,309.0,340827,1103,320.0,352960,3.816600,1103,309.0,340827,1103,320.0,352960,3.816600,1,309.0,309,1,309000.0,320.0,320,320000.0,1103,309.0,340827,1103,320.0,352960,309.0,309,1,320.0,320,1,0,0
59314,59315,774,21,122,39,Female,Other,320,909,1.61,2,4,613.0,2452,341.791359,4,280.0,1120,46.188022,2484,8,909.0,3843252,4228,320.0,1352960,3.935110,650,909.0,590850,650,320.0,208000,3.832486,2,909.0,1818,2,909000.0,320.0,640,320000.0,650,909.0,590850,650,320.0,208000,909.0,909,1,320.0,320,0,0,0
38901,38902,10088,5,10,13,Male,Student,240,229,4.16,2,5,315.0,1575,117.760350,5,216.0,1080,32.863353,2638,31,229.0,2196797,9593,240.0,2302320,4.389029,1052,229.0,240908,1052,240.0,252480,4.446967,3,229.0,687,3,229000.0,240.0,720,240000.0,1052,229.0,240908,1052,240.0,252480,229.0,229,1,240.0,240,1,1,0


In [633]:
train = featured_df[featured_df['engagement_score'].notna()].copy()
test = featured_df[featured_df['engagement_score'].isna()].copy()
train.shape[0] == raw_train.shape[0], test.shape[0] == raw_test.shape[0]

(True, True)

In [634]:
train.sample(3)

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid,video_id_nunique_usr_cat,views_mean_usr_cat_x,views_sum_usr_cat_x,views_count_usr_cat_x,views_cv_usr_cat,followers_mean_usr_cat_x,followers_sum_usr_cat_x,followers_cv_usr_cat,user_id_nunique_usr_cat,views_mean_usr_cat_y,views_sum_usr_cat_y,views_count_usr_cat_y,followers_mean_usr_cat_y,followers_sum_usr_cat_y,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat,male,student,working
13086,13087,16229,4,64,22,Male,Student,230,781,4.18,1,4,781.000000,3124,0.000000,4,230.000000,920,0.000000,1781,5,781.0,3513719,4499,230.0,1034770,4.078661,910,781.0,710710,910,230.0,209300,4.390034,4,781.0,3124,4,781000.0,230.0,920,230000.0,910,781.0,710710,910,230.0,209300,781.0,781,1,230.0,230,1,1,0
2540,2541,5175,5,109,32,Male,Working Professional,240,229,3.16,2,3,358.333333,1075,224.011904,3,226.666667,680,23.094011,2638,31,229.0,2196797,9593,240.0,2302320,4.389029,1053,229.0,241137,1053,240.0,252720,4.680070,2,229.0,458,2,229000.0,240.0,480,240000.0,1053,229.0,241137,1053,240.0,252720,229.0,229,1,240.0,240,1,0,1
7543,7544,7693,15,32,43,Female,Working Professional,340,662,2.98,3,5,617.600000,3088,159.551872,5,290.000000,1450,48.476799,1322,3,662.0,1545108,2334,340.0,793560,4.228355,801,662.0,530262,801,340.0,272340,4.135329,2,662.0,1324,2,662000.0,340.0,680,340000.0,801,662.0,530262,801,340.0,272340,662.0,662,1,340.0,340,0,0,1


In [635]:
drop_col = [
            'row_id',
            'user_id','category_id','video_id',
            'gender','profession',
            #'usr_rank',
            #'cat_rank','video_rank',
]

In [636]:
sub_train = train.drop(drop_col,axis=1)
sub_train.head(3)

,age,followers,views,engagement_score,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid,video_id_nunique_usr_cat,views_mean_usr_cat_x,views_sum_usr_cat_x,views_count_usr_cat_x,views_cv_usr_cat,followers_mean_usr_cat_x,followers_sum_usr_cat_x,followers_cv_usr_cat,user_id_nunique_usr_cat,views_mean_usr_cat_y,views_sum_usr_cat_y,views_count_usr_cat_y,followers_mean_usr_cat_y,followers_sum_usr_cat_y,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat,male,student,working
0,24,180,1000,4.33,2,3,744.666667,2234,221.125153,3,246.666667,740,57.735027,635,2,1000.0,962000,962,180.0,173160,3.618930,503,1000.0,503000,503,180.0,90540,3.654689,1,1000.0,1000,1,1000000.0,180.0,180,180000.0,503,1000.0,503000,503,180.0,90540,1000.0,1000,1,180.0,180,1,1,0
1,14,330,714,1.79,3,5,602.800000,3014,130.273558,5,308.000000,1540,34.928498,1507,4,714.0,2239818,3137,330.0,1035210,3.994371,807,714.0,576198,807,330.0,266310,4.005123,2,714.0,1428,2,714000.0,330.0,660,330000.0,807,714.0,576198,807,330.0,266310,714.0,714,1,330.0,330,0,1,0
2,19,180,138,4.35,1,4,138.000000,552,0.000000,4,180.000000,720,0.000000,2020,6,138.0,608856,4412,180.0,794160,4.913325,644,138.0,88872,644,180.0,115920,4.602973,4,138.0,552,4,138000.0,180.0,720,180000.0,644,138.0,88872,644,180.0,115920,138.0,138,1,180.0,180,1,1,0


In [637]:
# corr = sub_train.corr().round(2)
# plt.figure(figsize=(15,8))
# sns.heatmap(corr,annot=False)

In [638]:
x_train = sub_train.drop(['engagement_score'],axis=1)
y_train = sub_train['engagement_score']

scaler = StandardScaler()
train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(train_scaled,index=x_train.index,columns=x_train.columns)
x_train.head(3)

,age,followers,views,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid,video_id_nunique_usr_cat,views_mean_usr_cat_x,views_sum_usr_cat_x,views_count_usr_cat_x,views_cv_usr_cat,followers_mean_usr_cat_x,followers_sum_usr_cat_x,followers_cv_usr_cat,user_id_nunique_usr_cat,views_mean_usr_cat_y,views_sum_usr_cat_y,views_count_usr_cat_y,followers_mean_usr_cat_y,followers_sum_usr_cat_y,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat,male,student,working
0,-0.094759,-1.572002,1.850629,0.127028,-0.662592,1.140598,0.242153,0.612823,-0.662592,-0.155670,-0.674876,1.447580,-1.371838,-0.647159,1.850629,-0.658821,-1.142798,-1.572002,-1.240651,-0.990462,-0.808649,1.870338,0.589774,-0.808649,-1.595095,-1.238268,-0.923498,-1.016455,1.850629,-0.199301,-1.016455,1.850629,-1.572002,-1.220020,-1.572002,-0.822112,1.850629,0.651489,-0.822112,-1.572002,-1.309602,1.850629,1.850629,0.0,-1.572002,-1.572002,0.838051,0.999115,-0.49789
1,-1.211393,1.682203,0.785722,1.774618,0.543186,0.473025,1.010811,-0.079251,0.543186,1.544197,1.153583,0.481294,-0.223567,-0.395254,0.785722,0.322796,-0.315185,1.682203,0.053194,-0.451916,0.329429,0.794599,0.913102,0.329429,1.709164,1.150807,-0.463548,-0.324912,0.785722,0.404606,-0.324912,0.785722,1.682203,0.117317,1.682203,0.395625,0.785722,0.990522,0.395625,1.682203,1.307286,0.785722,0.785722,0.0,1.682203,1.682203,-1.193244,0.999115,-0.49789
2,-0.653076,-1.572002,-1.358987,-1.520563,-0.059703,-1.714156,-1.415390,-1.071628,-0.059703,-2.003352,-0.720588,-0.998587,0.451963,-0.143349,-1.358987,-0.930105,0.169968,-1.572002,-0.308596,0.866266,-0.280791,-1.371923,-1.239499,-0.280791,-1.595095,-0.893302,0.321142,1.058174,-1.358987,-0.831428,1.058174,-1.358987,-1.572002,0.284484,-1.572002,-0.257306,-1.358987,-1.266639,-0.257306,-1.572002,-0.931741,-1.358987,-1.358987,0.0,-1.572002,-1.572002,0.838051,0.999115,-0.49789


In [639]:
r2_scorer = make_scorer(r2_score)

In [640]:
def cross_validation(x_train,y_train,model,model_name,cv=5,scoring=r2_scorer):
    scores = cross_val_score(model, x_train, y_train, cv=5,scoring=scoring)
    print(f"{model_name}: mean r2_score {scores.mean():.2f} with a standard deviation of {scores.std():.2f}")
    return

In [641]:
dummy_model = DummyRegressor()
cross_validation(x_train,y_train,dummy_model,'Dummy Model')

Dummy Model: mean r2_score -0.00 with a standard deviation of 0.00


In [642]:
linear_model = LinearRegression()
cross_validation(x_train,y_train,linear_model,'Linear Model')

Linear Model: mean r2_score 0.33 with a standard deviation of 0.01


In [643]:
# svm_model = SVR(kernel='poly',degree=1)
# cross_validation(x_train,y_train,svm_model,'SVM Model')

In [644]:
knn_model = KNeighborsRegressor()
cross_validation(x_train,y_train,knn_model,'KNN Model')

KNN Model: mean r2_score 0.30 with a standard deviation of 0.01


In [645]:
rf_model =  RandomForestRegressor(max_depth=9, random_state=100)
cross_validation(x_train,y_train,rf_model,'Random Forest Model')

Random Forest Model: mean r2_score 0.37 with a standard deviation of 0.01


In [646]:
ada_boost_model =  AdaBoostRegressor(random_state=100)
cross_validation(x_train,y_train,ada_boost_model,'AdaBoot')

AdaBoot: mean r2_score 0.27 with a standard deviation of 0.00


In [647]:
gbr_model =  GradientBoostingRegressor(max_depth=5, random_state=100,subsample=0.9,learning_rate=0.05)
cross_validation(x_train,y_train,gbr_model,'Gradient boosting')

Gradient boosting: mean r2_score 0.37 with a standard deviation of 0.01


In [648]:
gbr_model =  HistGradientBoostingRegressor(max_depth=7, random_state=100,learning_rate=0.05)
cross_validation(x_train,y_train,gbr_model,'Gradient boosting')

Gradient boosting: mean r2_score 0.37 with a standard deviation of 0.01


In [649]:
xgb_model =  xgb.XGBRegressor(n_estimators=100,max_depth=11, random_state=100,learning_rate=0.05,
                              subsample=1,colsample_bytree =1,objective='reg:squarederror')
cross_validation(x_train,y_train,xgb_model,'XGB')

XGB: mean r2_score 0.39 with a standard deviation of 0.01


In [650]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.05,
    'feature_fraction': 0.95,
    'bagging_fraction': 0.95,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 15,
    "num_leaves": 128,  
    "max_bin": 512,
    "n_estimators": 200,
    "random_state":345
}
lgb_model = lgb.LGBMRegressor(**hyper_params)
cross_validation(x_train,y_train,lgb_model,'LGB')

LGB: mean r2_score 0.39 with a standard deviation of 0.00


In [651]:
model = xgb_model.fit(x_train,y_train)

In [652]:
x_test = test[x_train.columns].copy()
scaled_test = scaler.transform(x_test)
x_test = pd.DataFrame(scaled_test,index=x_test.index,columns=x_test.columns)
x_test.head(3)

,age,followers,views,category_id_nunique_usr,video_id_nunique_usr,views_mean_usr,views_sum_usr,views_std_usr,views_count_usr,followers_mean_usr,followers_sum_usr,followers_std_usr,user_id_nunique_cat,video_id_nunique_cat,views_mean_cat,views_sum_cat,views_count_cat,followers_mean_cat,followers_sum_cat,engagement_score_cv_cat,user_id_nunique_vid,views_mean_vid,views_sum_vid,views_count_vid,followers_mean_vid,followers_sum_vid,engagement_score_cv_vid,video_id_nunique_usr_cat,views_mean_usr_cat_x,views_sum_usr_cat_x,views_count_usr_cat_x,views_cv_usr_cat,followers_mean_usr_cat_x,followers_sum_usr_cat_x,followers_cv_usr_cat,user_id_nunique_usr_cat,views_mean_usr_cat_y,views_sum_usr_cat_y,views_count_usr_cat_y,followers_mean_usr_cat_y,followers_sum_usr_cat_y,views_mean_usr_cat,views_sum_usr_cat,views_count_usr_cat,followers_mean_usr_cat,followers_sum_usr_cat,male,student,working
89197,-1.211393,-1.572002,-1.358987,-1.520563,-0.662592,-1.714156,-1.551384,-1.071628,-0.662592,-2.003352,-1.131991,-0.998587,0.451963,-0.143349,-1.358987,-0.930105,0.169968,-1.572002,-0.308596,0.866266,1.916749,-1.371923,-0.881682,1.916749,-1.595095,0.542835,1.110855,0.366631,-1.358987,-1.026146,0.366631,-1.358987,-1.572002,-0.217017,-1.572002,2.094049,-1.358987,-0.891441,2.094049,-1.572002,0.641339,-1.358987,-1.358987,0.0,-1.572002,-1.572002,0.838051,0.999115,-0.497890
89198,-1.211393,-0.487267,1.254877,1.774618,1.748965,1.145527,3.184735,-0.385340,1.748965,1.084916,2.296370,1.440581,0.002926,-0.269302,1.254877,1.169766,0.005206,-0.487267,-0.126973,-2.358143,0.258299,1.268526,1.291750,0.258299,-0.493675,-0.005469,-2.463413,0.366631,1.254877,1.945417,0.366631,1.254877,-0.487267,0.200901,-0.487267,0.319517,1.254877,1.387562,0.319517,-0.487267,0.040752,1.254877,1.254877,0.0,-0.487267,-0.487267,0.838051,0.999115,-0.497890
89199,2.138509,0.597468,0.465505,0.127028,1.146076,0.591607,1.753848,-1.071628,1.146076,1.322476,1.747832,0.314164,1.799074,0.360461,0.465505,2.311080,1.416525,0.597468,1.730337,0.657424,0.074859,0.471126,0.417911,0.074859,0.607744,0.343575,0.734263,1.058174,0.465505,1.934129,1.058174,0.465505,0.597468,1.398932,0.597468,0.123237,0.465505,0.471277,0.123237,0.597468,0.423079,0.465505,0.465505,0.0,0.597468,0.597468,-1.193244,-1.000886,2.008476


In [653]:
x_test['engagement_score'] = model.predict(x_test)
x_test['engagement_score'].head(3)

89197    4.108878
89198    3.812982
89199    2.759950
Name: engagement_score, dtype: float32

In [654]:
test['engagement_score'] = x_test['engagement_score']
test[['row_id','engagement_score']].to_csv("submission_4_xgb.csv",index=False)